In [17]:
from deuces import Card, Evaluator
import itertools as it
import operator as op
from datetime import timedelta, datetime
import pathos.multiprocessing as mp
from pathos.multiprocessing import Pool
import log
import numpy as np
import sys
from hurry.filesize import size
import array
from bisect import bisect_left, bisect_right
import scipy.sparse as sp
import operator as op
import random

In [14]:
card_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 
         83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 
         179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239]

suits = list('hdcs')
ranks = list('A23456789TJQK')
card_labels = ['%s%s' % (r, s) for s, r in it.product(suits, ranks)]


def dfh(obj):
    return size(sys.getsizeof(obj))


def gen_hand5_scores(card_primes, card_labels):
    cards = map(Card.new, card_labels)
    e = Evaluator()
    
    pool = mp.Pool(mp.cpu_count())
    
    ratings = pool.map(lambda h: e.evaluate(h, ()), it.combinations(cards, 5))
    max_rating = max(ratings)
    scores = pool.map(lambda r: max_rating-r, ratings)
    ids = pool.map(lambda h: reduce(op.mul, h), it.combinations(card_primes, 5))
    
    hand5_scores = dict(zip(ids, scores))
    
    return hand5_scores
        

def gen_hand7_scores(card_primes, hand5_scores):
    def score_max(card_ids):
        hand_ids = [reduce(op.mul, h) for h in it.combinations(card_ids, 5)]
        scores = [hand5_scores[hand_id] for hand_id in hand_ids]
        max_index = np.argmax(scores)
        
        return hand_ids[max_index], scores[max_index] 
    
    pool = mp.Pool(mp.cpu_count())
    
    hand7_scores = pool.map(lambda h: score_max(h), it.combinations(card_primes, 7))
    hand7_scores = dict(hand7_scores)
    
    return hand7_scores

In [114]:
start = datetime.now()
hand5_scores = gen_hand5_scores(card_primes, card_labels)
finish = datetime.now()

len(hand5_scores), dfh(hand5_scores), str(finish-start)

(2598960, '96M', '0:01:17.288858')

In [ ]:
start = datetime.now()
hand7_scores = gen_hand7_scores(card_primes, hand5_scores)
finish = datetime.now()

len(hand7_scores), str(finish-start)

In [130]:
def get_hand5s(hand_board):
    i = 1
    j = 0
    cards = []
    hand = hand_board
    while i < 7:
        card_prime = card_primes[j]
        if (hand % card_prime) == 0:
            i += 1
            hand /= card_prime
            for c in cards:
                yield hand5_scores[hand_board / (c*card_prime)]
            cards.append(card_prime)
        j += 1
    for c in cards:
        yield hand5_scores[hand_board / (c*hand)]

expected_hand = [11, 59, 89, 167, 193, 223, 233]
expected_hands = [hand5_scores[reduce(op.mul, h)] for h in it.combinations(expected_hand, 5)]
expected_hand_prime = reduce(op.mul, expected_hand)

start = datetime.now()
for counter in range(1000000):
    actual_hands = [c for c in get_hand5s(expected_hand_prime)]
finish = datetime.now()
print str(finish-start)
if set(expected_hands) != set(actual_hands):
    raise Exception('Error!')

0:00:12.122865


In [129]:
def get_hand5_score(hand_board):
    hands = [hand5_scores[reduce(op.mul, h)] for h in it.combinations(hand_board, 5)]
    return max(hands)

expected_hand = [11, 59, 89, 167, 193, 223, 233]

start = datetime.now()
for counter in range(1000000):
    max_score = get_hand5_score(expected_hand)
finish = datetime.now()
print str(finish-start)

0:00:13.593251


In [175]:
def get_hand5s(hand, board):
    cards = hand + board
    cards_prime = reduce(op.mul, cards)
    l = 6
    scores = []
    while l > 0:
        c = cards.pop()
        scores += [hand5_scores[cards_prime/(c*ci)] for ci in cards]
        l -= 1
        
    return max(scores)

hand, board = [17, 59], [89, 167, 193, 13, 233]
expected_score = max([hand5_scores[reduce(op.mul, h)] for h in it.combinations(hand+board, 5)])

start = datetime.now()
for counter in range(1000000):
    actual_score = get_hand5s(hand, board)
finish = datetime.now()
print str(finish-start), expected_score, actual_score

0:00:07.373697 763 763
